# Battle between 2 Neighbourhoods - Melbourne & Sydney, Australia

## Part 1 - The Data Frame


In [1]:
# import required libraries

import pandas as pd # library for data analsysis

import requests # library to handle requests
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2018.12=py37_0
  - defaults/win-64::blaze==0.11.3=py37_0
  - defaults/win-64::numba==0.41.0=py37hf9181ef_0
done

## Package Plan ##

  environment location: C:\Users\Chima-Seamfix\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2020.02  |           py37_0           6 KB
    anaconda-custom            |           py37_1           2 KB
    argh-0.26.2                |        py37_1001          33 KB  conda-forge
    autopep8-1.5               |             py_0          38 KB  conda-forge
    bcrypt-3.1.7               |   py37h8055547_1          40 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         184 KB  c

DEBUG menuinst_win32:__init__(196): Menu: name: 'Anaconda${PY_VER} ${PLATFORM}', prefix: 'C:\Users\Chima-Seamfix\Anaconda3', env_name: 'None', mode: 'user', used_mode: 'user'
DEBUG menuinst_win32:create(320): Shortcut cmd is %windir%\System32\WindowsPowerShell\v1.0\powershell.exe, args are ['-ExecutionPolicy', 'ByPass', '-NoExit', '-Command', '"& \'C:\\Users\\Chima-Seamfix\\Anaconda3\\shell\\condabin\\conda-hook.ps1\' ; conda activate \'C:\\Users\\Chima-Seamfix\\Anaconda3\' "']


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Chima-Seamfix\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.1-py_0
  folium             conda-forge/noarch::folium-0.5.0-py_0
  vincent            con

### Retrieve data for Manchester and perform the data preparation

In [6]:
# retrieve all London tables from the wiki page 
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_areas_of_London")

# convert the required table (first table in the list) to a dataframe
df_london = pd.DataFrame(data[1])

# rename the columns
df_london.columns = ['Neighbourhood','Borough','City','pd','dc','os']

# drop the unneeded columns
df_london.drop(['pd','dc','os'], axis=1, inplace=True)

# update City to be London for all rows
df_london['City'] = 'London'

# reorder columns
df_london = df_london[['City','Borough','Neighbourhood']]

df_london

,City,Borough,Neighbourhood
0,London,London borough,Location
1,London,"Bexley, Greenwich [7]",Abbey Wood
2,London,"Ealing, Hammersmith and Fulham[8]",Acton
3,London,Croydon[8],Addington
4,London,Croydon[8],Addiscombe
5,London,Bexley,Albany Park
6,London,Redbridge[9],Aldborough Hatch
7,London,City[10],Aldgate
8,London,Westminster[10],Aldwych
9,London,Brent[11],Alperton


In [7]:
df_london.shape

(534, 3)

## Add Longitude and Latitude to the DF

In [11]:
import geocoder

In [16]:
# create two new columns for latitude and lonitude
df_london['Latitude'] = ""
df_london['Longitude'] = ""

# set row counter to 0
i = 0

# loop through all rows of the dataframe to fetch latitude and longitude and add to dataframe.

for N in df_london['Neighbourhood']:
    
    lat_lng_coords = None
    neighbourhood = df_london.iloc[i]['Neighbourhood'] #retrieve the first neighbourhood in the datafame
    borough = df_london.iloc[i]['Borough'] #retrieve the first Borough in the datafame
    lookup = neighbourhood + ","  + borough + ", England"
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis(lookup)
        lat_lng_coords = g.latlng
   
    df_london['Latitude'][i] = lat_lng_coords[0] #update latitude column
    df_london['Longitude'][i] = lat_lng_coords[1] #update longitude column
   
    print(df_london.iloc[[i]])
     
    i = i+1 #increment the row counter
    
# check the dataframe has updated correctly
df_london.head()

     City         Borough Neighbourhood Latitude Longitude
0  London  London borough      Location  50.9055   -2.4295
     City                Borough Neighbourhood Latitude Longitude
1  London  Bexley, Greenwich [7]    Abbey Wood  53.0512  -1.39859
     City                            Borough Neighbourhood Latitude Longitude
2  London  Ealing, Hammersmith and Fulham[8]         Acton  51.4951 -0.220839
     City     Borough Neighbourhood Latitude  Longitude
3  London  Croydon[8]     Addington  51.3588 -0.0329062
     City     Borough Neighbourhood Latitude  Longitude
4  London  Croydon[8]    Addiscombe  51.3736 -0.0903331
     City Borough Neighbourhood Latitude Longitude
5  London  Bexley   Albany Park  51.4357   0.12588
     City       Borough     Neighbourhood Latitude Longitude
6  London  Redbridge[9]  Aldborough Hatch   54.092  -1.38166
     City   Borough Neighbourhood Latitude  Longitude
7  London  City[10]       Aldgate  51.5221 -0.0870944
     City          Borough Neighbourho

      City      Borough   Neighbourhood Latitude  Longitude
68  London  Bromley[35]  Bromley Common  51.3882  0.0360982
      City                                           Borough Neighbourhood  \
69  London  Kensington and ChelseaHammersmith and Fulham[36]      Brompton   

   Latitude Longitude  
69  51.3911   0.53632  
      City    Borough Neighbourhood Latitude Longitude
70  London  Brent[36]   Brondesbury  51.5536  -0.21173
      City     Borough   Neighbourhood Latitude Longitude
71  London  Barnet[37]  Brunswick Park -33.2317    115.83
      City      Borough Neighbourhood Latitude  Longitude
72  London  Enfield[38]   Bulls Cross  51.6756 -0.0594072
      City     Borough Neighbourhood Latitude Longitude
73  London  Barnet[39]     Burnt Oak  51.6021  -0.26642
      City     Borough   Neighbourhood Latitude  Longitude
74  London  Barnet[39]  Burroughs, The  51.3734  0.0444103
      City        Borough Neighbourhood Latitude Longitude
75  London  Southwark[40]    Camberwell  51.

       City  Borough Neighbourhood Latitude Longitude
138  London  Bromley   Derry Downs  51.3856   0.11657
       City Borough Neighbourhood Latitude Longitude
139  London   Brent   Dollis Hill   51.561  -0.23437
       City  Borough Neighbourhood Latitude Longitude
140  London  Bromley         Downe  51.4057   0.01506
       City   Borough Neighbourhood Latitude  Longitude
141  London  Lewisham       Downham  51.4483 -0.0202873
       City    Borough Neighbourhood Latitude Longitude
142  London  Southwark       Dulwich  51.5054  -0.08921
       City Borough Neighbourhood Latitude Longitude
143  London  Ealing        Ealing  51.5141  -0.30073
       City                 Borough Neighbourhood Latitude Longitude
144  London  Kensington and Chelsea   Earls Court  51.4975  -0.19761
       City     Borough Neighbourhood Latitude Longitude
145  London  Wandsworth    Earlsfield  51.4423  -0.18823
       City Borough Neighbourhood Latitude Longitude
146  London  Barnet   East Barnet  51.6443 

       City Borough Neighbourhood Latitude Longitude
211  London  Camden     Hampstead  51.5341 -0.139177
       City Borough            Neighbourhood Latitude Longitude
212  London  Barnet  Hampstead Garden Suburb  51.5782 -0.196931
       City               Borough Neighbourhood Latitude Longitude
213  London  Richmond upon Thames       Hampton  51.4266  -0.37798
       City               Borough Neighbourhood Latitude Longitude
214  London  Richmond upon Thames  Hampton Hill  51.4266  -0.37798
       City               Borough Neighbourhood Latitude Longitude
215  London  Richmond upon Thames  Hampton Wick   51.416  -0.31371
       City Borough Neighbourhood Latitude Longitude
216  London  Ealing       Hanwell  51.5088   -0.3363
       City   Borough Neighbourhood Latitude Longitude
217  London  Hounslow      Hanworth  51.4688 -0.359483
       City     Borough Neighbourhood Latitude Longitude
218  London  Hillingdon     Harefield  51.5326 -0.448425
       City Borough Neighbourhood 

       City   Borough Neighbourhood Latitude   Longitude
282  London  Lewisham           Lee  51.4626 -0.00984194
       City Borough   Neighbourhood Latitude Longitude
283  London  Bexley  Lessness Heath  51.4857  0.145185
       City   Borough Neighbourhood Latitude  Longitude
284  London  Lewisham      Lewisham  51.4483 -0.0202873
       City         Borough Neighbourhood Latitude  Longitude
285  London  Waltham Forest        Leyton  51.5941 -0.0126308
       City         Borough Neighbourhood Latitude  Longitude
286  London  Waltham Forest   Leytonstone  51.5941 -0.0126308
       City        Borough Neighbourhood Latitude  Longitude
287  London  Tower Hamlets     Limehouse  51.5113 -0.0389912
       City      Borough Neighbourhood Latitude Longitude
288  London  Westminster  Lisson Grove  51.5226 -0.165442
       City Borough  Neighbourhood Latitude  Longitude
289  London  Newham  Little Ilford  51.5516  0.0588549
       City      Borough  Neighbourhood Latitude Longitude
290  Lond

       City  Borough Neighbourhood Latitude Longitude
353  London  Bromley     Orpington  51.3788   0.10185
       City Borough Neighbourhood Latitude Longitude
354  London  Barnet        Osidge  51.6517  -0.19746
       City   Borough Neighbourhood Latitude Longitude
355  London  Hounslow      Osterley  51.4672  -0.36653
       City  Borough Neighbourhood Latitude Longitude
356  London  Lambeth          Oval  51.4537 -0.118373
       City      Borough Neighbourhood Latitude  Longitude
357  London  Westminster    Paddington   51.494 -0.0961895
       City  Borough  Neighbourhood Latitude  Longitude
358  London  Enfield  Palmers Green   51.663 -0.0477422
       City        Borough Neighbourhood Latitude Longitude
359  London  Brent, Ealing    Park Royal  51.5277  -0.26747
       City                 Borough  Neighbourhood Latitude Longitude
360  London  Hammersmith and Fulham  Parsons Green  51.4742 -0.200346
       City    Borough Neighbourhood Latitude Longitude
361  London  Southwark

       City  Borough Neighbourhood Latitude  Longitude
425  London  Bromley  Southborough   51.396  0.0457962
       City     Borough Neighbourhood Latitude Longitude
426  London  Wandsworth   Southfields  51.4543 -0.199809
       City  Borough Neighbourhood Latitude Longitude
427  London  Enfield     Southgate  51.6339  -0.13289
       City        Borough Neighbourhood Latitude Longitude
428  London  Tower Hamlets  Spitalfields  51.5184  -0.07468
       City Borough Neighbourhood Latitude Longitude
429  London  Merton     St Helier  51.3985 -0.187018
       City      Borough Neighbourhood Latitude Longitude
430  London  Westminster    St James's  51.4912  -0.13482
       City               Borough Neighbourhood Latitude Longitude
431  London  Richmond upon Thames  St Margarets  51.4281 -0.271584
       City Borough Neighbourhood Latitude Longitude
432  London  Camden      St Giles   50.953   -4.0889
       City   Borough Neighbourhood Latitude  Longitude
433  London  Lewisham      St 

       City    Borough Neighbourhood Latitude  Longitude
495  London  Greenwich     Well Hall  51.4744  0.0555739
       City Borough Neighbourhood Latitude Longitude
496  London  Bexley       Welling   51.462   0.11171
       City Borough Neighbourhood Latitude Longitude
497  London   Brent       Wembley   51.553  -0.29553
       City Borough Neighbourhood Latitude Longitude
498  London   Brent  Wembley Park  51.5616 -0.281969
       City   Borough Neighbourhood Latitude Longitude
499  London  Havering    Wennington  54.1254    -2.584
       City                 Borough  Neighbourhood Latitude Longitude
500  London  Kensington and Chelsea  West Brompton  51.5012 -0.192222
       City     Borough Neighbourhood Latitude Longitude
501  London  Hillingdon  West Drayton  51.5262 -0.439465
       City Borough Neighbourhood Latitude Longitude
502  London  Ealing   West Ealing  51.5135  -0.31971
       City   Borough Neighbourhood Latitude Longitude
503  London  Haringey    West Green   51.11

,City,Borough,Neighbourhood,Latitude,Longitude
0,London,London borough,Location,50.9055,-2.4295
1,London,"Bexley, Greenwich [7]",Abbey Wood,53.0512,-1.39859
2,London,"Ealing, Hammersmith and Fulham[8]",Acton,51.4951,-0.220839
3,London,Croydon[8],Addington,51.3588,-0.0329062
4,London,Croydon[8],Addiscombe,51.3736,-0.0903331


### Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [17]:
# get the geographical coordinates of England
address = 'England, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of England are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of England are 52.7954791, -0.5402402866174321.


In [20]:
# create map of England using latitude and longitude values
map_england = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, label in zip(df_london['Latitude'], df_london['Longitude'], df_london['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_england)  
    
map_england

#### Clean up the data

In [22]:
df_clean = df_london[( (df_london['Latitude']>=51.2) & (df_london['Latitude']<=51.7) & (df_london['Longitude']>=-0.6) & (df_london['Longitude']<=0.2) )]

print(df_clean.shape)
print('There are {} neighbourhoods.'.format(len(df_clean['Neighbourhood'].unique())))



(463, 5)
There are 458 neighbourhoods.


In [23]:
# create map of England using latitude and longitude values
map_england = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, label in zip(df_clean['Latitude'], df_clean['Longitude'], df_clean['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_england)  
    
map_england

In [24]:
df_london = df_clean

## London Neighbourhood Clutter

#### Get the nearby venues for each neighbourhood using the foursquare API

Initialize Foursquare credentials

In [27]:
CLIENT_ID = 'EDQWCUSYPKVAOKLC4RPLEUCCSR43CCDYTM0RY44VR1RI5UQL' # your Foursquare ID
CLIENT_SECRET = 'P5ZNEZXCS5G4I3I4TAKVIEDZUHHO31DXDS1HYOMNI5RCF4DB' # your Foursquare Secret
ACCESS_TOKEN = 'IUFLVLJLQGWOZKXFWWIO0I1QM1DILGHK3ZQ5GRAUNBJIJUPI'
VERSION = '20180604'
LIMIT = 300
RADIUS = 2000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EDQWCUSYPKVAOKLC4RPLEUCCSR43CCDYTM0RY44VR1RI5UQL
CLIENT_SECRET:P5ZNEZXCS5G4I3I4TAKVIEDZUHHO31DXDS1HYOMNI5RCF4DB


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
london1 = df_london.loc[(df_london['City'] == "London") & (df_london.index <= 300),['Neighbourhood']]
london2 = df_london.loc[(df_london['City'] == "London") & (df_london.index > 300) & (df_london.index <= 500),['Neighbourhood']]
london3 = df_london.loc[(df_london['City'] == "London") & (df_london.index > 500),['Neighbourhood']]

In [37]:

venues1 = getNearbyVenues(names=london1['Neighbourhood'],
                        latitudes=df_london['Latitude'],
                        longitudes=df_london['Longitude']
                        )



Acton
Addington
Addiscombe
Albany Park
Aldgate
Angel
Arkley
Arnos Grove
Balham
Bankside
Barking
Barkingside
Barnehurst
Barnes
Barnes Cray
Barnet Gate
Barnet (also Chipping Barnet, High Barnet)
Barnsbury
Battersea
Beckenham
Beckton
Becontree
Becontree Heath
Bedford Park
Belmont
Belmont
Belsize Park
Belvedere
Bermondsey
Bethnal Green
Bexley (also Old Bexley, Bexley Village)
Bexleyheath (also Bexley New Town)
Bickley
Biggin Hill
Blackfen
Blackheath
Blackheath Royal Standard
Blackwall
Blendon
Bloomsbury
Botany Bay
Bounds Green
Bow
Bowes Park
Brentford
Brent Cross
Brimsdown
Brixton
Brockley
Bromley
Bromley (also Bromley-by-Bow)
Bromley Common
Brondesbury
Bulls Cross
Burnt Oak
Burroughs, The
Camberwell
Cambridge Heath
Camden Town
Canary Wharf
Cann Hall
Canning Town
Carshalton
Castelnau
Castle Green
Catford
Chadwell Heath
Chalk Farm
Charing Cross
Chase Cross
Cheam
Chelsea
Chelsfield
Chessington
Childs Hill
Chinatown
Chinbrook
Chingford
Chislehurst
Chiswick
Church End
Clapham
Clerkenwell
Cockf

In [39]:
venues2 = getNearbyVenues(names=london2['Neighbourhood'],
                        latitudes=df_london['Latitude'],
                        longitudes=df_london['Longitude']
                        )


Marks Gate
Maryland
Marylebone (also St Marylebone)
Mayfair
Maze Hill
Merton Park
Mile End
Mill Hill
Millbank
Millwall
Monken Hadley
Morden
Morden Park
Mortlake
Motspur Park
Mottingham
Muswell Hill
Nag's Head
Neasden
New Addington
New Barnet
New Cross
New Eltham
New Malden
New Southgate
Newbury Park
Newington
Nine Elms
Norbiton
Norbury
North Cray
North End
North Finchley
North Harrow
North Kensington
North Sheen
North Woolwich
Northolt
Northumberland Heath
Northwood
Norwood Green
Notting Hill
Nunhead
Oakleigh Park
Old Coulsdon
Old Ford
Old Malden
Old Oak Common
Orpington
Osidge
Osterley
Oval
Paddington
Palmers Green
Park Royal
Parsons Green
Peckham
Penge
Pentonville
Perivale
Petersham
Petts Wood
Pimlico
Pinner
Plaistow
Plaistow
Ponders End
Poplar
Pratt's Bottom
Primrose Hill
Purley
Putney
Ratcliff
Rayners Lane
Raynes Park
Richmond
Riddlesdown
Roehampton
Rotherhithe
Ruislip
Rush Green
Ruxley
Sanderstead
Sands End
Selhurst
Selsdon
Seven Kings
Seven Sisters
Shacklewell
Shadwell
Shepherd's

In [42]:
venues3 = getNearbyVenues(names=london3['Neighbourhood'],
                        latitudes=df_london['Latitude'],
                        longitudes=df_london['Longitude']
                        )

West Drayton
West Ealing
West Hackney
West Ham
West Hampstead
West Harrow
West Hendon
West Kensington
West Norwood
West Wickham
Westcombe Park
Westminster
Whetstone
White City
Whitechapel
Widmore (also Widmore Green)
Whitton
Willesden
Wimbledon
Winchmore Hill
Wood Green
Woodford
Woodford Green
Woodlands
Woodside
Woolwich
Worcester Park
Wormwood Scrubs
Yeading
Yiewsley


In [43]:

# union the dataframes together
venues = pd.concat([venues1, venues2, venues3]).reset_index(drop=True)

# check data looks correct - there should be 554 neighbourhoods
print('There are {} neighbourhoods.'.format(len(venues['Neighbourhood'].unique())))
print(venues.shape)

There are 458 neighbourhoods.
(35502, 7)


### Analyze each of the neighbourhoods in London

In [44]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighbourhood'] = venues['Neighbourhood'] 

# move neighborhood column to the first column
onehot = onehot[['Neighbourhood'] + [col for col in onehot.columns if col != 'Neighbourhood']]

onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group the rows by neighbourhood

In [46]:
grouped = onehot.groupby('Neighbourhood').mean().reset_index()
grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Acton,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.00,0.00,0.01,0.00,0.0,0.000000,0.0,0.00,0.0,0.00
1,Addington,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.00,0.04,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.00
2,Addiscombe,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.00
3,Albany Park,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.00
4,Aldgate,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.01,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.00
5,Angel,0.0,0.01,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.00,0.00,0.00,0.02,0.0,0.000000,0.0,0.01,0.0,0.00
6,Arkley,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.00
7,Arnos Grove,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.00
8,Balham,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.000000,0.01,...,0.00,0.00,0.00,0.00,0.0,0.000000,0.0,0.02,0.0,0.00
9,Bankside,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.00


### The the top 10 venues for each neighbourhood in a new dataframe

In [47]:
# create a function to sort the venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
# create the new dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = grouped['Neighbourhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Pub,Coffee Shop,Café,Park,Persian Restaurant,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place,Bakery
1,Addington,Tram Station,Park,Grocery Store,Gas Station,Golf Course,Business Service,Fish & Chips Shop,Fast Food Restaurant,Supermarket,Scenic Lookout
2,Addiscombe,Pub,Coffee Shop,Park,Indian Restaurant,Italian Restaurant,Grocery Store,Clothing Store,Fast Food Restaurant,Café,Bookstore
3,Albany Park,Grocery Store,Pub,Coffee Shop,Fast Food Restaurant,Hotel,Beer Bar,Train Station,Pharmacy,Platform,Lake
4,Aldgate,Coffee Shop,Hotel,Garden,Cocktail Bar,Gym / Fitness Center,Art Gallery,Steakhouse,Street Food Gathering,Beer Bar,Pub


### Clutters

In [49]:
kclusters = 4

grouped_clustering = grouped.drop('Neighbourhood', 1) #drop the neighborhood column

# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters = kclusters, n_init = 12)
kmeans.fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([1, 2, 3, 3, 0, 0, 3, 3, 1, 0, 2, 2, 3, 1, 3, 3, 3, 1, 1, 3, 3, 2,
       2, 1, 3, 1, 2, 0, 3, 3, 3, 3, 1, 3, 1, 1, 3, 3, 0, 1, 1, 3, 1, 3,
       1, 3, 0, 1, 3, 3, 3, 1, 3, 3, 1, 1, 0, 1, 3, 3, 0, 3, 1, 2, 3, 2,
       1, 0, 3, 3, 0, 3, 1, 1, 0, 2, 3, 3, 1, 3, 1, 0, 1, 3, 3, 1, 3, 3,
       3, 1, 3, 0, 3, 0, 3, 3, 1, 1, 1, 3, 3, 1, 3, 1, 1, 3, 2, 1, 1, 1,
       1, 3, 3, 3, 3, 0, 1, 0, 1, 1, 0, 1, 1, 3, 1, 3, 2, 3, 3, 1, 0, 3,
       2, 3, 3, 3, 3, 3, 2, 1, 3, 3, 1, 1, 0, 3, 3, 1, 2, 1, 3, 3, 1, 1,
       1, 3, 1, 1, 3, 2, 3, 3, 2, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3,
       3, 3, 1, 3, 3, 3, 3, 3, 0, 1, 3, 2, 3, 3, 0, 1, 3, 1, 1, 3, 3, 1,
       1, 3, 3, 0, 0, 1, 1, 3, 2, 1, 1, 3, 1, 3, 0, 3, 1, 3, 0, 1, 1, 1,
       3, 3, 1, 1, 2, 1, 0, 1, 1, 3, 3, 1, 1, 1, 1, 2, 3, 3, 3, 1, 0, 3,
       0, 3, 0, 1, 2, 0, 1, 1, 1, 2, 3, 3, 0, 0, 3, 3, 1, 0, 2, 2, 3, 1,
       3, 3, 3, 1, 1, 3, 3, 2, 2, 1, 3, 3, 1, 2, 0, 3, 3, 3, 3, 1, 3, 1,
       1, 3, 3, 0, 1, 1, 3, 1, 1, 3, 3, 0, 1, 3, 3,

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labelss', kmeans.labels_)

df_merged = df_london

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

df_merged#.head() # check the last columns!

,City,Borough,Neighbourhood,Latitude,Longitude,Cluster Labelss,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,London,"Ealing, Hammersmith and Fulham[8]",Acton,51.4951,-0.220839,1,1,Pub,Coffee Shop,Café,Park,Persian Restaurant,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place,Bakery
3,London,Croydon[8],Addington,51.3588,-0.0329062,2,2,Tram Station,Park,Grocery Store,Gas Station,Golf Course,Business Service,Fish & Chips Shop,Fast Food Restaurant,Supermarket,Scenic Lookout
4,London,Croydon[8],Addiscombe,51.3736,-0.0903331,3,3,Pub,Coffee Shop,Park,Indian Restaurant,Italian Restaurant,Grocery Store,Clothing Store,Fast Food Restaurant,Café,Bookstore
5,London,Bexley,Albany Park,51.4357,0.12588,3,3,Grocery Store,Pub,Coffee Shop,Fast Food Restaurant,Hotel,Beer Bar,Train Station,Pharmacy,Platform,Lake
7,London,City[10],Aldgate,51.5221,-0.0870944,0,0,Coffee Shop,Hotel,Garden,Cocktail Bar,Gym / Fitness Center,Art Gallery,Steakhouse,Street Food Gathering,Beer Bar,Pub
11,London,Islington[8],Angel,51.532,-0.10626,0,0,Pub,Coffee Shop,Bakery,Pizza Place,Mediterranean Restaurant,Theater,Cocktail Bar,Beer Bar,Hotel,Bookstore
15,London,Barnet[12],Arkley,51.6485,-0.22951,3,3,Pub,Coffee Shop,Supermarket,Bookstore,Hardware Store,Steakhouse,Park,Campground,Sports Club,Mini Golf
16,London,Enfield[12],Arnos Grove,51.6224,-0.127234,3,3,Grocery Store,Pub,Coffee Shop,Café,Greek Restaurant,Park,Pizza Place,Fast Food Restaurant,Bakery,Gym / Fitness Center
17,London,Wandsworth[13],Balham,51.4745,-0.132392,1,1,Café,Coffee Shop,Italian Restaurant,Pub,Park,Pizza Place,Bar,Bakery,Market,Supermarket
18,London,Southwark[14],Bankside,51.5082,-0.09523,0,0,Coffee Shop,Theater,Hotel,Scenic Lookout,Gym / Fitness Center,Grocery Store,Cocktail Bar,Pub,Art Gallery,Indie Movie Theater


### Visualize the clusters

In [56]:
# how many neighbourhoods are in each cluster?

df_visual = df_merged[['City','Cluster Labels','Neighbourhood']].groupby(['City','Cluster Labels'], as_index=False).count()
df_visual

,City,Cluster Labels,Neighbourhood
0,London,0,53
1,London,1,168
2,London,2,42
3,London,3,200


In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
df_merged.loc[(df_merged['Cluster Labels'] == 0) & (df_merged['City'] == "London"), df_merged.columns[[2] + list(range(6, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Aldgate,0,Coffee Shop,Hotel,Garden,Cocktail Bar,Gym / Fitness Center,Art Gallery,Steakhouse,Street Food Gathering,Beer Bar,Pub
11,Angel,0,Pub,Coffee Shop,Bakery,Pizza Place,Mediterranean Restaurant,Theater,Cocktail Bar,Beer Bar,Hotel,Bookstore
18,Bankside,0,Coffee Shop,Theater,Hotel,Scenic Lookout,Gym / Fitness Center,Grocery Store,Cocktail Bar,Pub,Art Gallery,Indie Movie Theater
42,Bermondsey,0,Coffee Shop,Hotel,Theater,Scenic Lookout,Grocery Store,Cocktail Bar,Park,Gym / Fitness Center,Italian Restaurant,Garden
55,Bloomsbury,0,Coffee Shop,Hotel,Pizza Place,History Museum,Ice Cream Shop,Steakhouse,Theater,Bookstore,Liquor Store,Restaurant
64,Brixton,0,Hotel,Pub,Theater,Café,Park,Scenic Lookout,Garden,Plaza,Art Museum,Coffee Shop
76,Cambridge Heath,0,Coffee Shop,Pub,Hotel,Café,Italian Restaurant,Bakery,Restaurant,Beer Bar,Cocktail Bar,Bookstore
80,Canning Town,0,Hotel,Coffee Shop,Grocery Store,Café,Pub,Sandwich Place,Supermarket,Hotel Bar,Gym / Fitness Center,Restaurant
88,Charing Cross,0,Hotel,Theater,Clothing Store,Bookstore,Plaza,Cocktail Bar,Ice Cream Shop,Pub,Beer Bar,Scenic Lookout
92,Chelsea,0,Hotel,Ice Cream Shop,Café,Plaza,Exhibit,Art Gallery,Boutique,Science Museum,Cocktail Bar,Park


In [64]:
df_merged.loc[(df_merged['Cluster Labels'] == 1) & (df_merged['City'] == "London"), df_merged.columns[[2] + list(range(6, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Acton,1,Pub,Coffee Shop,Café,Park,Persian Restaurant,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place,Bakery
17,Balham,1,Café,Coffee Shop,Italian Restaurant,Pub,Park,Pizza Place,Bar,Bakery,Market,Supermarket
23,Barnes,1,Pub,Café,Italian Restaurant,Park,Hotel,Garden,Gastropub,Coffee Shop,Deli / Bodega,German Restaurant
27,Barnsbury,1,Pub,Café,Coffee Shop,Park,Burger Joint,Mediterranean Restaurant,Gastropub,Art Gallery,Bakery,Theater
28,Battersea,1,Pub,Coffee Shop,Park,Café,Italian Restaurant,French Restaurant,Bakery,Cocktail Bar,Supermarket,Pizza Place
35,Bedford Park,1,Pub,Park,Coffee Shop,Café,Hotel,Burger Joint,Pizza Place,Supermarket,Italian Restaurant,Gym / Fitness Center
40,Belsize Park,1,Café,Bakery,Deli / Bodega,Thai Restaurant,French Restaurant,Bookstore,Gastropub,Cocktail Bar,Coffee Shop,Pub
48,Biggin Hill,1,Pub,Café,Park,Coffee Shop,Grocery Store,Supermarket,Gastropub,Bakery,Pizza Place,Bar
51,Blackheath,1,Pub,Café,Coffee Shop,Garden,Park,Turkish Restaurant,Indie Movie Theater,Market,Supermarket,Street Food Gathering
52,Blackheath Royal Standard,1,Pub,Park,Bar,Garden,Grocery Store,Coffee Shop,Indian Restaurant,Brewery,Turkish Restaurant,Art Gallery


In [65]:
df_merged.loc[(df_merged['Cluster Labels'] == 2) & (df_merged['City'] == "London"), df_merged.columns[[2] + list(range(6, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Addington,2,Tram Station,Park,Grocery Store,Gas Station,Golf Course,Business Service,Fish & Chips Shop,Fast Food Restaurant,Supermarket,Scenic Lookout
20,Barking,2,Grocery Store,Supermarket,Soccer Field,Park,Pub,Fast Food Restaurant,Clothing Store,Go Kart Track,Rugby Pitch,Chinese Restaurant
21,Barkingside,2,Grocery Store,Pizza Place,Indian Restaurant,Park,Supermarket,Thai Restaurant,Soccer Field,Pub,Steakhouse,Golf Course
32,Becontree,2,Grocery Store,Supermarket,Park,Train Station,Bus Stop,Chinese Restaurant,Portuguese Restaurant,English Restaurant,Liquor Store,Martial Arts Dojo
33,Becontree Heath,2,Grocery Store,Supermarket,Soccer Field,Park,Pub,Fast Food Restaurant,Clothing Store,Go Kart Track,Rugby Pitch,Chinese Restaurant
41,Belvedere,2,Grocery Store,Construction & Landscaping,Supermarket,Pub,Train Station,Historic Site,Hardware Store,Fast Food Restaurant,English Restaurant,Eastern European Restaurant
84,Castle Green,2,Grocery Store,Supermarket,Soccer Field,Park,Pub,Fast Food Restaurant,Clothing Store,Go Kart Track,Rugby Pitch,Chinese Restaurant
86,Chadwell Heath,2,Grocery Store,Supermarket,Soccer Field,Park,Pub,Fast Food Restaurant,Clothing Store,Go Kart Track,Rugby Pitch,Chinese Restaurant
97,Chinbrook,2,Grocery Store,Park,Convenience Store,Pub,Pet Store,Café,Motorcycle Shop,Fish & Chips Shop,Fast Food Restaurant,Fried Chicken Joint
132,Dagenham,2,Grocery Store,Supermarket,Soccer Field,Park,Pub,Fast Food Restaurant,Clothing Store,Go Kart Track,Rugby Pitch,Chinese Restaurant


In [66]:
df_merged.loc[(df_merged['Cluster Labels'] == 3) & (df_merged['City'] == "London"), df_merged.columns[[2] + list(range(6, df_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Addiscombe,3,Pub,Coffee Shop,Park,Indian Restaurant,Italian Restaurant,Grocery Store,Clothing Store,Fast Food Restaurant,Café,Bookstore
5,Albany Park,3,Grocery Store,Pub,Coffee Shop,Fast Food Restaurant,Hotel,Beer Bar,Train Station,Pharmacy,Platform,Lake
15,Arkley,3,Pub,Coffee Shop,Supermarket,Bookstore,Hardware Store,Steakhouse,Park,Campground,Sports Club,Mini Golf
16,Arnos Grove,3,Grocery Store,Pub,Coffee Shop,Café,Greek Restaurant,Park,Pizza Place,Fast Food Restaurant,Bakery,Gym / Fitness Center
22,Barnehurst,3,Pub,Grocery Store,Supermarket,Electronics Store,Chinese Restaurant,Italian Restaurant,Clothing Store,Train Station,Furniture / Home Store,Café
24,Barnes Cray,3,Pub,Grocery Store,Supermarket,Hotel,Pet Store,Train Station,Clothing Store,Italian Restaurant,Pharmacy,Chinese Restaurant
25,Barnet Gate,3,Pub,Supermarket,Hotel,Italian Restaurant,Coffee Shop,Hardware Store,Steakhouse,Sandwich Place,Mini Golf,Seafood Restaurant
26,"Barnet (also Chipping Barnet, High Barnet)",3,Pub,Coffee Shop,Grocery Store,Café,Fast Food Restaurant,Pharmacy,Bookstore,Golf Course,Pizza Place,Soccer Stadium
30,Beckenham,3,Pub,Coffee Shop,Clothing Store,Pizza Place,Indian Restaurant,Gym / Fitness Center,Burger Joint,Café,Park,Hotel
31,Beckton,3,Grocery Store,Indian Restaurant,Supermarket,Coffee Shop,Fast Food Restaurant,Hotel,Sandwich Place,Pub,Furniture / Home Store,Discount Store
